# OpenSearch 한글 형태소 분석기 통한 키워드 검색 
>이 노트북은 SageMaker Studio* Data Science 3.0 kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.



여기서는 OpenSearch 가 설치된 것을 가정하고, 한글 형태소 분석기의 사용하는 법을 알려 드립니다.

---

### [중요]
- 이 노트북은 Bedrock Titan Embedding Model 을 기본으로 사용합니다. KoSIMCSERoberta 를 세이지 메이커 엔드포인트로 사용하신다면 아래의 선수 조건을 확인하세요.

#### 선수조건 (KoSIMCSERoberta 사용시)
- 임베딩 모델의 세이지 메이커 엔드포인트가 액티브 된 상태를 가정 합니다.
    - 세이지 메이커 엔드포인트에 배포하기 위해서는 아래 노트북을 실행하시고, Endpoint Name 만을 복사 하시면 됩니다.
    - [KoSIMCSERoberta Embedding Model 배포](https://github.com/gonsoomoon-ml/Kor-LLM-On-SageMaker/blob/main/1-Lab01-Deploy-LLM/4.Kor-Embedding-Model.ipynb)
    - SageMaker Endpoint 에 대해서는 공식 개발자 문서를 참조하세요 --> [Create your endpoint and deploy your model](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html)
- 오픈 서치 서비스가 액티브 된 상태를 가정 합니다.


---
## Ref: 
- [Amazon OpenSearch Service로 검색 구현하기](https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/blog-reindex)
- [OpenSearch Python Client](https://opensearch.org/docs/1.3/clients/python-high-level/)
- [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
- OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
    - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)
- [OpenSearch Query Description (한글)](https://esbook.kimjmin.net/05-search)


# 1. 환경 세팅

In [ ]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

## Bedrock Client 생성
### 선수 지식
아래의 노트북을 먼저 실행해서, Bedrock 에 접근 가능하게 합니다.
- amazon-bedrock-workshop-webinar-kr/00_Setup/setup.ipynb

In [ ]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

## Embedding 모델 선택

In [ ]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [ ]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):
    if is_bedrock_embeddings:

        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings

        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")
        
    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )        
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")
    
    return llm_emb

#### [중요] is_KoSimCSERobert == True 경우,  endpoint_name 을 꼭 넣어 주세요.

In [ ]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)    

# 2. 데이터 준비


##  신한은행 FAQ 데이터 세트로 구현
- [중요] 저자 및 동료가 아래의 웹사이트에서 크로링한 기준으로 구성 하였습니다.
- 인터넷뱅킹 FAQ > 스마트뱅킹 No.1 ~ N. 89 로 구성되었습니다. 
- https://www.shinhan.com/hpe/index.jsp#050101020000

In [ ]:
import pandas as pd
pd.options.display.max_rows = 20

data_file_path = "data/fsi_smart_faq_ko.csv"
df = pd.read_csv(data_file_path)
df

## 데이터 전처리
- 여기서 no 는 제거 합니다. 

In [ ]:
os.makedirs("data", exist_ok=True)

In [ ]:
def preprocess_data(df):

    ldf = df.copy()
    ldf.rename(columns={'Category': 'ask'}, inplace=True)
    df_index = ldf.drop(['no'], axis=1)
    df_index.to_csv("data/fsi_smart_faq_ko_preprocess.csv", index=None)

    return df_index

pre_df = preprocess_data(df)
pre_df.head(3)

### CSVLoader 로 문서 로딩

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, SpacyTextSplitter

In [ ]:
loader = CSVLoader(
    file_path="data/fsi_smart_faq_ko_preprocess.csv",
    # csv_args={
    #     "delimiter": ",",
    #     "fieldnames": ["Category", "Information", "type", "Source"],
    # },    
    source_column="Source",
    encoding="utf-8"
)
import time
documents_fsi = loader.load()


## 메타 데이터 생성
- 기존 Category 컬럼을 ask 로 변경 합니다.
- 컬럼의 type, source 는 metadata 로 생성하고, 내용에서는 삭제 합니다.
- 타임스탬프 및 임베딩 모델의 엔드포인트 이름을 metadata 로 추가 합니다.

In [ ]:
import time
documents_fsi = loader.load()

def create_metadata(docs):
    # # add a custom metadata field, such as timestamp
    for idx, doc in enumerate(docs):

        #print ("previous:", doc)
        # type 을 메타 데이타로 저장
        stype = doc.page_content.split("type: ")[1].split("\n")[0]
        split_content = doc.page_content.split("type: ")
        content = split_content[0]        
        metadata = split_content[1]                
        doc.metadata['type'] = metadata.split("\n")[0]        
        doc.page_content = content # metadata 제외하고 content 만 저장
        doc.metadata['timestamp'] = time.time()

create_metadata(documents_fsi)

In [ ]:
print (len(documents_fsi))
print (documents_fsi[0])

## Text Spliter 로 청킹
참고: 검색된 문서/텍스트는 질문에 대답하기에 충분한 정보를 포함할 만큼 커야 합니다. 하지만 LLM 프롬프트에 들어갈 만큼 충분히 작습니다. <BR>
또한 임베딩 모델에는 입력 토큰 길이는 KoSimCSERobert는 512개, titanEmbedding(8,912개) 토큰으로 제한되어 있습니다. <BR>
이 사용 사례를 위해 [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html)를 사용하여 500자가 겹치는 약 92자의 청크를 생성합니다.

In [ ]:
if is_bedrock_embeddings:
    chunk_size = 2048
    chunk_overlap = 50
elif is_KoSimCSERobert:
    chunk_size = 800 # This is maxumum
    chunk_overlap = 0


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

docs = text_splitter.split_documents(documents_fsi)
print(f"Number of documents after split and chunking={len(docs)}")

# 3. OpenSearch 벡터 Indexer 생성
### 선수 조건
- 랭체인 오프서처 참고 자료
    - [Langchain Opensearch](https://python.langchain.com/docs/integrations/vectorstores/opensearch)

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [ ]:
from utils.opensearch import opensearch_utils
http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

In [ ]:
index_name = "genai-demo-index-v1"
index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
    opensearch_utils.delete_index(os_client, index_name)
else:
    print("Index does not exist")

## 인덱스 생성

In [ ]:
from langchain.vectorstores import OpenSearchVectorSearch

In [ ]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=10000,
    timeout=60
)

## 인덱스 확인

In [ ]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

## 형태소 분석기 사용하기
- 영어권의 문자들과 다르게 한글, 일본어, 중국어 등은 단순한 공백만으로는 좋은 검색 결과를 얻기 힘듭니다.
- 출시하고라는 단어가 들어간 문서를 출시하고라는 정확히 같은 단어만으로 검색할 수 있다면 답답하겠죠?
- 출시하고라는 단어를 출시, 출시하고 등 다양하게 검색하기 위해서는 형태소 분석기가 필요합니다.
- OpenSearch 에서는 2개의 한국어 analyer를 제공하고 있습니다.
    - 은전한잎 (seunjeon_tokenizer)
        - https://catalog.us-east-1.prod.workshops.aws/workshops/de4e38cb-a0d9-4ffe-a777-bf00d498fa49/ko-KR/indexing/stemming#
    - Nori (nori_tokenizer)
        - 설명: https://esbook.kimjmin.net/06-text-analysis/6.7-stemming/6.7.2-nori
    - Sample 코드에서는 "Nori"를 기반으로 진행합니다.

### 인덱싱 수정하기 (형태소 분석기 사용 enablement)

In [ ]:
new_index_name = f'{index_name}-with-tokenizer'
new_index_name

### [TIP]
- **token filter**
    - Amazon OpenSearch Service에서는 나만의 필터를 토크나이저와 함께 구성해서 사용할 수 있습니다. 
    - nori_tokenizer를 사용하되, 원하는 캐릭터 필터와 토큰 필터 조합을 구성할 수 있습니다.
    - 문장이 입력되면 캐릭터 필터(char_filter), 토크나이저(tokenizer), 토큰 필터(filter) 순으로 동작하게 됩니다.
    - 입력문장이 "<b>Start</b> 이천이십삼년 韓國" 이것일 경우, 
    - 아래 샘플 코드의 옵션으로 사용한다면 <b>Start</b>의 html 코드가 html_strip에 의해 처리되고, nori_tokenizer로 토큰화됩니다.
    - 그 후에 nori_number는 이천이십삼을 2023으로, nori_readingform은 韓國을 한국으로, lowercase는 Start를 start로 처리합니다.
    - **토큰필터 참고자료**
        - https://esbook.kimjmin.net/06-text-analysis/6.6-token-filter
        - https://opensearch.org/docs/latest/analyzers/token-filters/index/
    
- **discard_punctuation**
    - true(기본값), false가 있으며 문장부호 또는 구두점을 어떻게 다룰지에 대한 설정을 할 수 있습니다.
- **decompound_mode**
    - none, discard(기본값), mixed가 있으며 복합명사를 어떻게 다룰지에 대한 설정을 할 수 있습니다.
    - none: 복합명사를 분리하지 않고 하나의 토큰으로 저장합니다.
    - discard: 복합명사를 분리하여 토큰으로 저장합니다.
    - mixed: 복합명사를 분리하지 않은 토큰과 분리한 토큰을 모두 저장합니다.
- **신조어, 업무용어, 상표**
     - 노리 토크나이저는 [mecab-ko-dic](https://bitbucket.org/eunjeon/mecab-ko-dic/src/master/) 을 사용하고 있지만 때로는 신조어, 업무 용어, 상표 등을 위한 사용자 사전이 필요할 수 있습니다.
     - user_dictionary_rules를 이용해서 사용자 사전을 만들 수 있고, 아래에서 사용자 사전을 적용했을 때와 적용하지 않았을 때를 비교해 볼 수 있습니다.
- **동의어, 불용어**
    - Amazon OpenSearch Service에서는 Package를 이용하면 사용자사전, 동의어, 불용어를 관리할 수 있습니다. 자세한 사항은 [이곳](https://docs.aws.amazon.com/ko_kr/opensearch-service/latest/developerguide/custom-packages.html)을 참조해 주십시오. 
    - S3에 사용자 사전을 업로드 후, Package에 등록합니다. 그리고 등록된 텍스트 사전을 원하는 OpenSearch 도메인에 연결합니다.
    - 아래와 같이 user_dictionary에 analyzers/<Package ID> 를 이용해서 적용할 수 있습니다. 이 Package ID는 OpenSearch 콘솔 화면에서 업로드한 사전의 상세 페이지에서 확인할 수 있습니다.


In [ ]:
tokenizer = "nori" #["nori", "seunjeon"]
analyzer_config = {
    "tokenizer": tokenizer, 
    "tokenizer_type": f'{tokenizer}_tokenizer',
    "char_filter": ["html_strip"],
    "filter": ["nori_number", "nori_readingform", "lowercase"],
    "decompound_mode": "mixed",
    "discard_punctuation": "true",
    "metadate_type": "keyword"
    #"user_dictionary_rules": ["c++", "워라밸", "먹방"],
    #"user_dictionary": "analyzers/F255700190"
}

In [ ]:
index_info[index_name]["settings"]["analysis"] = {
    "tokenizer": {
        analyzer_config["tokenizer"]: {
            "type": analyzer_config["tokenizer_type"],
            "decompound_mode": analyzer_config["decompound_mode"],
            "discard_punctuation": analyzer_config["discard_punctuation"],
            #"user_dictionary_rules": analyzer_config["user_dictionary_rules"],
            #"user_dictionary": analyzer_config["user_dictionary"],
        }
    },
    "analyzer": {
        "my_analyzer": {
            "type": "custom",
            "tokenizer": analyzer_config["tokenizer"],
            "char_filter": analyzer_config["char_filter"],
            "filter": analyzer_config["filter"],
        }
    }
}

# Setting for Columns to be adapted by Tokenizer (tokenizer가 적용될 컬럼에 맞춰서 수정)
index_info[index_name]["mappings"]["properties"]["text"]["analyzer"] = "my_analyzer"
index_info[index_name]["mappings"]["properties"]["text"]["search_analyzer"] = "my_analyzer"

# Setting for vector index column (변경 없음)
index_info[index_name]["settings"]["index"] = {
    "number_of_shards": "5",
    "knn.algo_param": {"ef_search": "512"},
    "knn": "true",
    "number_of_replicas": "2"
}
del index_info[index_name]["aliases"]
del index_info[index_name]["mappings"]["properties"]["metadata"]["properties"]["source"]["fields"]
del index_info[index_name]["mappings"]["properties"]["metadata"]["properties"]["type"]["fields"]
index_info[index_name]["mappings"]["properties"]["metadata"]["properties"]["source"]["type"] = analyzer_config["metadate_type"]
index_info[index_name]["mappings"]["properties"]["metadata"]["properties"]["type"]["type"] = analyzer_config["metadate_type"]

new_index_info = index_info[index_name]

In [ ]:
pprint(new_index_info)

### 형태소 분석기용 인덱서 생성

In [ ]:
index_exists = opensearch_utils.check_if_index_exists(os_client, new_index_name)
if index_exists:
    opensearch_utils.delete_index(os_client, new_index_name)
else:
    print("Index does not exist")

In [ ]:
opensearch_utils.create_index(
    os_client,
    index_name=new_index_name,
    index_body=new_index_info
)

### Re-indexing

In [ ]:
_reindex = {
    "source": {"index": index_name},
    "dest": {"index": new_index_name}
}
print("_reindex: \n", _reindex)

In [ ]:
os_client.reindex(_reindex)

# 6. 키워드 검색

### 'Text" 에 "약관, 뱅킹" 단어를 검색합니다.

* without tokenizer (index_name)

In [ ]:
query = "뱅킹"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

* with tokenizer (new_index_name)

In [ ]:
query = "뱅킹"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

## 형태소 분석 결과 확인
"약관" 또는 "뱅킹" 확인 <BR>
#### [중요]:  doc_id: 위의 문서 인덱스 정보 확인 후 수정

In [ ]:
doc_id = "[YOUR-DOC-ID]"
#doc_id = "29b3a9bf-7edf-4893-9791-d5c05f5f20b5" 

* without tokenizer

In [ ]:
os_client.termvectors(index=index_name, id=doc_id, fields='text')

* with tokenizer

In [ ]:
os_client.termvectors(index=new_index_name, id=doc_id, fields='text')

## Minimum_should_match 활용
- An optional parameter of type string that represents the minimum number of matching clauses for a document to be returned. This should only be used with “OR” operator, and is more flexible than a simple “and/or,” since users can set rules depending on the length of the phrase to be matched. 
    - **query에 있는 단어의 n%이상 존재하는 문서만 가져온다**
    - 75% (eg. 3 out of 4 words to be matched, or 6 out of 8)
    - 2  (minimum 2 words to be matched, irrespective of length of string)

min_shoud_match를 25, 75, 100 으로 변경해 보자
 - 숫자가 작아질 수록 검색 문서가 많아진다
 - 추가정보는 아래를 참고
     - [OpenSearch Match, Multi-Match, and Match Phrase Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-match-multi-match-and-match-phrase-queries/)
     - OpenSearch Query 에서 Filter, Must, Should, Not Mush 에 대한 설명 입니다.
         - [OpenSearch Boolean Queries](https://opster.com/guides/opensearch/opensearch-search-apis/opensearch-boolean-queries/#:~:text=Boolean%20queries%20are%20used%20to,as%20terms%2C%20match%20and%20query_string.)

##### minimum_should_match=25

In [ ]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=25
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

##### minimum_should_match=75

In [ ]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=75
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

##### minimum_should_match=100

In [ ]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=100
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

## Filter 활용
- document내 metadata를 활용하여 search space를 줄일 수 있다.
- 특히 filter의 경우 search 전에 수행되기 때문에, 검색 속도 향상을 기대할 수 있다
- syntax
    - filter=[{"term"**[고정]**: {"metadata.source"**[메타데이터 이름, 혹은 메타데이터 아니여도 상관없음]**: "신한은행"**[조건명]**}},]
    - list 형식으로 복수개 filter 설정 가능

In [ ]:
query = "인터넷 뱅킹으로 예적금 해약"
query = opensearch_utils.get_query(
    query=query,
    minimum_should_match=75,
    filter=[
        {"term": {"metadata.source": "신한은행"}},
        {"term": {"metadata.type": "인터넷뱅킹"}},
    ]
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, new_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

## tokenized index example (참고용)

In [ ]:
test_index_info = {
    "settings": {
        "index": {
            "number_of_shards": "5",
            "knn.algo_param": {"ef_search": "512"},
            "knn": "true",
            "number_of_replicas": "2"
        },
        "analysis": {
            "tokenizer": {
                "nori": {
                    "type": "nori_tokenizer"
                }
            },
            "analyzer": {
                "my_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "metadata": {
                "properties": {
                    "row": {"type": "long"},
                    "source": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    },
                    "timestamp": {"type": "float"},
                    "type": {
                        "type": "text",
                        "fields": {
                            "keyword": {
                                "type": "keyword",
                                "ignore_above": 256
                            }
                        }
                    }
                }
            },
            "text": {
                "type": "text",
                "analyzer": "my_analyzer",
                "search_analyzer": "my_analyzer",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {
                    "engine": "nmslib",
                    "space_type": "l2",
                    "name": "hnsw",
                    "parameters": {
                        "ef_construction": 512,
                        "m": 16
                    }
                }
            }
        }
    }
}